In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings


In [13]:
loader = PyPDFLoader("Data/PG-Manual.pdf")
pages = loader.load_and_split()

In [14]:
pages[0]

Document(metadata={'source': 'Data/PG-Manual.pdf', 'page': 0}, page_content='(1) POSTGRADUATE PROGRAMMES \nProcedures and Requirements \n  \n \n \n \n \n \n \n \nIndian Institute of Technology Kanpur \nApproved in Senate (2015-16/1st) meeting on September 02, 2015 & \n  afterward updates various Senate meetings approval (upto 06 February 2024) \n \n \n \nWeb site:   http://www.iitk.ac.in/spgc')

In [15]:
cleaned_pages = []

for page in pages:
    # Remove newline characters
    page_text = page.page_content
    page_text = page_text.replace("\n", "")
    # Remove tab characters
    page_text = page_text.replace("\t", "")
    # Add more cleaning operations if necessary
    
    # Append cleaned page to the list
    cleaned_pages.append(page_text)

In [ ]:
cleaned_pages[35]

'(35) 5.  If the M.Tech / M. Des thesis option is exercised, the problem which forms the major content of the M Tech./ M.Des thesis should be different from the one reported in the PhD thesis. To ensure this, there should be plagiarism check performed between the MTech / M.Des and the PhD thesis. Also, it is recommended that at least one member of the thesis oral boards should be common across the two boards. 6.    Student can deliver open seminar after M. Tech / M.Des defense without waiting for six months. 7.    For MBA programme kindly refer to Annexure-V.   11.7.1 Award of MSR degree with PhD degree to those students who are admitted to PhD program in Engineering directly after a B.Tech./BS(4Yr)/M.Sc. degree or have converted from MSR degree to PhD degree:   All students who are pursuing the PhD degree directly after B.Tech./BS(4Yr)/M.Sc. degree can opt to get the additional MSR degree awarded along with the PhD degree at the time of graduation. This is subject to the following con

In [ ]:
import chromadb
chroma_client = chromadb.Client()

In [ ]:
collection = chroma_client.create_collection(name="my_collection")

In [ ]:
collection.add(
    documents=cleaned_pages,ids=[str(i) for i in range(len(cleaned_pages))])

In [ ]:
results = collection.query(
    query_texts="what are the minimum number of credits required by MTech student from department of mechanical engineering(ME) to get his degree",
    n_results=2
)
results

{'ids': [['20', '6']],
 'distances': [[0.7523107528686523, 0.8059523105621338]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['(21) S. No. Programme Minimum Total Credits Credits Through Course Work (Minimum)  Credits Through Research (Minimum) Minimum Duration3&4 Maximum Duration5 1 D.IIT. 72 36 18 2 Sem 3 Sem 2 M.Tech./M.Des. 144 541 63 4 Sem 4 Years 3 M.S. by Research students with B.Tech., B.S. (4-year program) or M.Sc.  144 36 72 4 Sem 4 Years 4 M.S. by Research students with B.Sc. 216 72 72 6 Sem 5 Years 5 M.B.A. 216 216 - 4 Sem 3 Years 6 Ph.D. (Design/Management/ Engineering except AE/BSBE/CSE/EE/ME/MSE/MSP) students with B.Des., B.Tech, B. Arch., B.S. (4-year program), M.Sc., M.A., M.Com. or Chartered Accountancy  216 722 72 6 Sem 7 Years 7 Ph.D. (AE/EE/ME/MSE/MSP) students with B.S. (4-year program), B.Tech. or M.Sc.  216 902 72 6 Sem 7 Years 8 Ph.D. (CSE/BSBE) students with B.S. (4-year program), B.Tech., M.Sc., M.C.A., B.Pharm., B.V.Sc. or M.B.B.S.  216

In [ ]:
from langchain.llms import CTransformers
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
prompt_template="""
Based on the given releated information of question try to answer the question given below.If you don't know the answer
return I dont know, don't to generate your own answer
information :{context}
question:{question}
return the helpful answer that set , dont return anything else
Helpful answer :"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML", model_type="llama",config={"temperature":0.7,'context_length' : 2048},device="cuda")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
output =LLMChain(llm=llm,prompt=PROMPT,output_key="OUTPUT",verbose=True)

In [ ]:
question="what are the minimum CPI required by MTech student from department of mechanical engineering(ME) to get his degree"

In [ ]:
results = collection.query(
    query_texts=question,
    n_results=2
)
results

{'ids': [['6', '23']],
 'distances': [[0.7449835538864136, 0.8692521452903748]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [["(7)  2.3.4. M.Des. The applicant must have a bachelor's degree in engineering or a 4-year bachelor's degree in science, Design or Architecture with marks/CPI not below the specified minimum. Candidates holding degrees in Fine Arts or Interior Design are not eligible to apply.  2.3.5. M.B.A. The applicant must have a Bachelor's degree in any discipline with minimum 60 percent marks/6.0 CPl. Candidates with Mathematics as a subject at 10+2 or equivalent level.  2.3.6. Ph.D. in Engineering and IDPs with M.Tech. program The applicant must have a master's degree in engineering with marks/CPI not below the specified minimum (please refer to 2.3.b) .  Those applicants may also be considered who have a bachelor's degree in engineering or a 4-year bachelor's degree in science with  a) a minimum of 75 percent marks/7.5 CPI b) JRF/95 percentile or hi

In [ ]:
output({"context":results["documents"],"question":question})



> Entering new LLMChain chain...
Prompt after formatting:

Based on the given releated information of question try to answer the question given below.If you don't know the answer
return I dont know, don't to generate your own answer
information :[["(7)  2.3.4. M.Des. The applicant must have a bachelor's degree in engineering or a 4-year bachelor's degree in science, Design or Architecture with marks/CPI not below the specified minimum. Candidates holding degrees in Fine Arts or Interior Design are not eligible to apply.  2.3.5. M.B.A. The applicant must have a Bachelor's degree in any discipline with minimum 60 percent marks/6.0 CPl. Candidates with Mathematics as a subject at 10+2 or equivalent level.  2.3.6. Ph.D. in Engineering and IDPs with M.Tech. program The applicant must have a master's degree in engineering with marks/CPI not below the specified minimum (please refer to 2.3.b) .  Those applicants may also be considered who have a bachelor's degree in engineering or a 4-year 

{'context': [["(7)  2.3.4. M.Des. The applicant must have a bachelor's degree in engineering or a 4-year bachelor's degree in science, Design or Architecture with marks/CPI not below the specified minimum. Candidates holding degrees in Fine Arts or Interior Design are not eligible to apply.  2.3.5. M.B.A. The applicant must have a Bachelor's degree in any discipline with minimum 60 percent marks/6.0 CPl. Candidates with Mathematics as a subject at 10+2 or equivalent level.  2.3.6. Ph.D. in Engineering and IDPs with M.Tech. program The applicant must have a master's degree in engineering with marks/CPI not below the specified minimum (please refer to 2.3.b) .  Those applicants may also be considered who have a bachelor's degree in engineering or a 4-year bachelor's degree in science with  a) a minimum of 75 percent marks/7.5 CPI b) JRF/95 percentile or higher in GATE for students from non-CFTIs . or a master’s degree in science or an allied area while satisfying each of the following cr

In [ ]:
from support import db

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [ ]:
docs = retriever.invoke("on which blockchain does diamante works")

In [1]:
from support import chain

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
ans = chain.invoke({"input":"on what blockchain does diamante work"})

In [7]:
ans["answer"].replace("?\nAnswer: ","")

'Diamante works on the Ethereum blockchain network as it is built on top of Ethereum.'

In [9]:
import torch
print("CUDA Available: ", torch.cuda.is_available())
print("CUDA Device Name: ", torch.cuda.get_device_name(0))

CUDA Available:  True
CUDA Device Name:  NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma